In [ ]:
!apt-get update
!apt-get install -y build-essential cmake git
!apt-get install -y libopencv-dev
!pip install opencv-python

In [ ]:
%%writefile image_denoising.cpp
#include <opencv2/opencv.hpp>
#include <iostream>
#include <vector>
#include <filesystem>
#include <chrono>
#include <string>
#include <cstdlib>

namespace fs = std::filesystem;
using namespace cv;
using namespace std;

bool endsWith(const string& str, const string& suffix) {
    if (str.length() >= suffix.length()) {
        return (0 == str.compare(str.length() - suffix.length(), suffix.length(), suffix));
    } else {
        return false;
    }
}

double psnr(const Mat& I1, const Mat& I2) {
    Mat s1;
    absdiff(I1, I2, s1);
    s1.convertTo(s1, CV_32F);
    s1 = s1.mul(s1);
    Scalar s = sum(s1);
    double sse = s.val[0] + s.val[1] + s.val[2];
    if (sse <= 1e-10) return 0;
    double mse = sse / (I1.channels() * I1.total());
    double psnr = 10.0 * log10((255 * 255) / mse);
    return psnr;
}

void processImage(const string& imagePath, const string& outputPath, int ksize, double sigma, double& totalInferenceTime, double& totalPSNR, int& imageCount) {
    Mat image = imread(imagePath, IMREAD_COLOR);
    if (image.empty()) {
        cerr << "Error loading image: " << imagePath << endl;
        return;
    }

    if (image.rows > 720 || image.cols > 1280) {
        cerr << "Image size exceeds the allowed dimensions of 1280x720p" << endl;
        return;
    }

    Mat result;
    auto start = chrono::high_resolution_clock::now();
    GaussianBlur(image, result, Size(ksize, ksize), sigma);
    auto end = chrono::high_resolution_clock::now();
    double inference_time = chrono::duration<double, milli>(end - start).count();

    imwrite(outputPath, result);

    double psnrValue = psnr(image, result);

    totalInferenceTime += inference_time;
    totalPSNR += psnrValue;
    ++imageCount;
}

void printGPUInfo() {
    cout << "GPU Metrics:" << endl;
    system("nvidia-smi --query-gpu=memory.used,utilization.gpu --format=csv,nounits,noheader | awk -F', ' '{print \"GPU Memory Usage (MB): \" $1 \" MB\"; print \"GPU Load (%): \" $2}'");
}

void processDataset(const string& rootFolder, int ksize, double sigma) {
    vector<string> noisyFolders = {"noisy5", "noisy15", "noisy35"};
    vector<string> denoisedFolders = {"denoised5", "denoised15", "denoised35"};

    double totalInferenceTime = 0;
    double totalPSNR = 0;
    int imageCount = 0;

    auto startTotalTime = chrono::high_resolution_clock::now();

    for (size_t i = 0; i < noisyFolders.size(); ++i) {
        string inputFolder = rootFolder + "/" + noisyFolders[i];
        string outputFolder = rootFolder + "/" + denoisedFolders[i];

        if (!fs::exists(outputFolder)) {
            fs::create_directory(outputFolder);
        }

        for (const auto& entry : fs::directory_iterator(inputFolder)) {
            string imagePath = entry.path().string();
            if (endsWith(imagePath, ".png") || endsWith(imagePath, ".jpg") || endsWith(imagePath, ".jpeg")) {
                string outputPath = outputFolder + "/" + entry.path().filename().string();
                processImage(imagePath, outputPath, ksize, sigma, totalInferenceTime, totalPSNR, imageCount);
            }
        }
    }

    auto endTotalTime = chrono::high_resolution_clock::now();
    double totalTimeTaken = chrono::duration<double, milli>(endTotalTime - startTotalTime).count();

    if (imageCount > 0) {
        double avgInferenceTime = totalInferenceTime / imageCount;
        double avgPSNR = totalPSNR / imageCount;

        cout << "Average Inference Time (ms): " << avgInferenceTime << endl;
        cout << "Average PSNR (dB): " << avgPSNR << endl;

        printGPUInfo();
    } else {
        cout << "No images processed." << endl;
    }
}

int main() {
    string rootFolder = "/content/drive/MyDrive/dataset";
    int ksize = 5;
    double sigma = 1.0;

    processDataset(rootFolder, ksize, sigma);

    return 0;
}


In [ ]:
!g++ -o image_denoising image_denoising.cpp `pkg-config --cflags --libs opencv4`
!./image_denoising